# Weekly speed averages

In [1]:
import altair as alt
import pandas as pd

from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis import styleguide
from segment_speed_utils.project_vars import SEGMENT_GCS

In [2]:
SPEEDS_SHAPE = "speeds_by_shape_peak_daytype"
SPEEDS_ROUTE = "speeds_by_route_direction_peak_daytype"

months = ["apr", "oct"]

df = pd.concat(
    [pd.read_parquet(
            f"{SEGMENT_GCS}rollup/{SPEEDS_SHAPE}_{m}2023.parquet"
        ) for m in months
    ], axis=0, ignore_index=True)

In [3]:
def base_bar_by_time_bin(df: pd.DataFrame, y_col: str) -> alt.Chart:
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x=alt.X("stop_sequence:O", title = "Stop Sequence",
                    axis=alt.Axis(labels=False)),
             y=alt.Y(f"{y_col}:Q", scale = alt.Scale(domain=[0, 45])),
             color=alt.Color(
                 "peak_offpeak:N", 
                scale = alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS[2:])
            ),
             tooltip = ["organization_name", "stop_sequence", 
                        "stop_id", y_col]
         ).properties(
             width=350, height=250,
        )#.facet("peak_offpeak:N", columns=1)
        .interactive()
    )
    return chart
    

def chart_by_time_bin(
    df: pd.DataFrame,
    y_col: str,
    shape_id: str
) -> alt.Chart:
    
    subset_df = df[df.shape_id==shape_id]
    
    weekday_df = subset_df[
        subset_df.weekday_weekend=="weekday"].reset_index(drop=True)
    weekend_df = subset_df[
        subset_df.weekday_weekend=="weekend"].reset_index(drop=True)
    
    weekday_chart = (base_bar_by_time_bin(weekday_df, y_col)
                     .properties(title = f"{shape_id} Weekday Speeds by Stop")
                    )
    
    weekend_chart = (base_bar_by_time_bin(weekend_df, y_col)
                     .properties(title = f"{shape_id} Weekend Speeds by Stop")
                    )
 
    chart = alt.vconcat(weekday_chart, weekend_chart)
    
    return chart

In [ ]:
# https://stackoverflow.com/questions/58919888/multiple-selections-in-altair

def make_interactive_legend(df):
    # Create selections
    selection_first_name = alt.selection_multi(
        fields=['First name'], empty='none')
    selection_last_name = alt.selection_multi(
        fields=['Last name'], empty='none')
    
    legend_first_name = alt.Chart(df).mark_point(size=100).encode(
        y=alt.Y('First name:N', axis=alt.Axis(orient='right')),
        color=alt.condition(selection_first_name,
                            alt.Color('First name:N', legend=None),
                            alt.value('lightgray') ),
    ).add_selection(
        selection_first_name

    )

# Create interactive model name legend
legend_last_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('Last name:N', axis=alt.Axis(orient='right')),
    shape=alt.Shape('Last name:N', legend=None),
    color=alt.condition(selection_last_name,
                        alt.value('black'),
                        alt.value('lightgray') ),
).add_selection(
    selection_last_name
)

# Combine plotting elements
chart = scatter | legend_first_name | legend_last_name

In [ ]:
def dual_bar_chart(
    df, 
    month: int,
    control_field: str,
    y_col: str
):
    """
    """
    subset_df = df[df.month==month]
    
    weekday_df = subset_df[
        subset_df.weekday_weekend=="weekday"].reset_index(drop=True)
    weekend_df = subset_df[
        subset_df.weekday_weekend=="weekend"].reset_index(drop=True)
    
    # Column that controls the bar charts
    category_selector = alt.selection_multi(fields=[control_field])
    
    # Build first chart
    weekday_chart = (base_bar_by_time_bin(weekday_df, y_col)
                     .properties(title = f"{shape_id} Weekday Speeds by Stop")
                    )
    
    chart1 = (alt.Chart(weekday_df)
              .mark_bar()
              .encode(
                x=alt.X(chart1_quant),
                y=alt.Y(chart1_nominal), 
        color = alt.Color(chart1_nominal, scale=alt.Scale(
        range=cp.CALITP_DIVERGING_COLORS), legend = None),
        tooltip = chart1_tooltip_cols)
        .properties(title=chart_title)
        .add_selection(category_selector))
    
    # Build second chart
    chart2 = (alt.Chart(df).mark_bar().encode(
        x=alt.X(chart2_quant),
        y=alt.Y(chart2_nominal), 
        color = alt.Color(chart2_nominal, scale=alt.Scale(
        range=cp.CALITP_DIVERGING_COLORS), legend = None),
        tooltip = chart2_tooltip_cols)
        .transform_filter(category_selector))
    
    chart1 = preset_chart_config(chart1)
    chart2 = preset_chart_config(chart2)
    return(chart1 & chart2)

In [4]:
operator = "Los Angeles County Metropolitan Transportation Authority"
test_shapes = ["1690148_JUNE23", "900359_DEC22"]
test = df[(df.organization_name==operator) & 
          (df.shape_id.isin(test_shapes))
         ].reset_index(drop=True)

In [19]:
input_dropdown = alt.binding_select(
    options=test.shape_id.unique().tolist(), 
    name='Shapes '
)
selection = alt.selection_point(fields=['shape_id'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('peak_offpeak:N').legend(None),
    alt.value('lightgray')
)


y_col = "p50_mph"

weekday_chart = (
    base_bar_by_time_bin(
        test[test.weekday_weekend=="weekday"], 
        y_col
    ).properties(title = f"Weekday Speeds by Stop")
    .add_params(selection)
    .transform_filter(selection)
)


legend = (
    alt.Chart(test[test.weekday_weekend=="weekday"])
    .mark_point()
    .encode(
        alt.Y('peak_offpeak:N').axis(orient='right'),
        color=color
    ).add_params(
        selection
    )
)

weekday_chart | legend

alt.HConcatChart(...)

In [ ]:
chart_by_time_bin(
    test, 
    "p50_mph",
    test_shapes[0]
)